In [2]:
import sympy as sp
import numpy as np
from sympy.physics.vector import dynamicsymbols
from sympy.physics.vector import time_derivative
from sympy.physics.vector import ReferenceFrame
N = ReferenceFrame('N')
import pylab as pl
import control
from EoM import *
from sympy.physics.mechanics import *
from numpy.linalg import matrix_rank, eig
import math
import intelligent_robotics as ir
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import WIP_utils as utils
import time as t
import Cal_joint as cj
import state_equation_pitch as sep
from dmpc import *


In [3]:
def rot_x(theta):
    rot_x = sp.Matrix([[1,0,0],
                       [0,sp.cos(theta),-sp.sin(theta)],
                       [0,sp.sin(theta),sp.cos(theta)]])
    return rot_x

def rot_y(theta):
    rot_y = sp.Matrix([[sp.cos(theta),0,sp.sin(theta)],
                       [0,1,0],
                       [-sp.sin(theta),0,sp.cos(theta)]])
    return rot_y

def rot_z(theta):
    rot_z = sp.Matrix([[sp.cos(theta),-sp.sin(theta),0],
                       [sp.sin(theta),sp.cos(theta),0],
                       [0,0,1]])
    return rot_z

def trans(a,b,c):
    x = sp.Matrix([[a],[b],[c]])
    return x

def get_torque_from_L(L,q,qd):
    round_L_round_q = sp.zeros(len(q),1);
    i = 0;
    for q_i in q:
        round_L_round_q_i = [];
        round_L_round_q_i = sp.diff(L,q_i);
        round_L_round_q[i] = round_L_round_q_i;
        i+=1;
     
    d_dt_round_L_round_qd = sp.zeros(len(qd),1);
    i = 0;
    for qd_i in qd:
        round_L_round_qd_i = [];
        d_dt_round_L_round_qd_i = [];
        round_L_round_qd_i = sp.diff(L,qd_i);
        d_dt_round_L_round_qd_i = time_derivative(round_L_round_qd_i,N);
        d_dt_round_L_round_qd[i] = d_dt_round_L_round_qd_i;
        i+=1;
        
    tau = d_dt_round_L_round_qd - round_L_round_q 
    return tau

In [5]:
r, L_1, L_b = sp.symbols('r, L_1, L_b')
L_1c = sp.symbols('L_1c')
m_w, m_1, m_b, g = sp.symbols('m_w, m_1, m_b, g')
I_w, I_1, I_b = sp.symbols('I_w, I_1, I_b')
theta_w, theta_1, theta_2 = dynamicsymbols('theta_w, theta_1, theta_2')

# Displacement Vector
x_c = trans(r*theta_w,0,0)
x_w = x_c +  trans(0,0,r)
x_1 = x_w +  rot_y(theta_1) @ trans(0,0,L_1c)
x_b = x_w +  rot_y(theta_1) @ trans(0,0,L_1)+  rot_y(theta_2) @ trans(0,0,L_b)

# Velocity Vector
v_w = sp.simplify(time_derivative(x_w,N))
v_1 = sp.simplify(time_derivative(x_1,N))
v_b = sp.simplify(time_derivative(x_b,N))

# Angular Velocity Vector
w_w = trans(0, theta_w.diff(), 0)
w_1 = trans(0, theta_1.diff(), 0)
w_b = trans(0, theta_2.diff(), 0)

# Generalized Coordinates
q = sp.Matrix([[theta_w], [theta_1], [theta_2]])
qd = q.diff()
qdd = qd.diff()

# Kinetic Energy
T_trans = 0.5*(m_w*v_w.dot(v_w)  + m_1*v_1.dot(v_1) + m_b*v_b.dot(v_b) )
T_wheel =I_w* w_w[1]**2 
T_1 =I_1* w_1[1]**2 
T_b =I_b* w_b[1]**2 

T_rot = 0.5*(T_wheel + T_1 +T_b)


# Potential Energy
V =  m_1*g*(L_1c*sp.cos(theta_1)) + m_b*g*(L_1*sp.cos(theta_1)+L_b*sp.cos(theta_2))

# Lagrangian
L = T_trans + T_rot - V


In [6]:
tau = get_torque_from_L(L,q,qd)

In [11]:
T_w, T_b= sp.symbols('T_w,T_b')
n, J_mb, K_tb, K_eb, b_mb, R_mb, V_b = sp.symbols('n, J_mb, K_tb, K_eb, b_mb, R_mb, V_b')
J_mw, K_tw, K_ew, b_mw, R_mw, V_w = sp.symbols('J_mw, K_tw, K_ew, b_mw, R_mw, V_w')
u = sp.Matrix([[T_w],[T_b]])
# u = sp.Matrix([[T_w], [a], [T_2], [T_3], [T_4]])
T_b = -n**2 * J_mb * (theta_2.diff().diff() - theta_1.diff().diff()) - n**2 * ((K_tb*K_eb)/R_mb + b_mb) * (theta_2.diff() - theta_1.diff()) + n*K_tb*V_b/R_mb
T_w = -J_mw * (theta_w.diff().diff() - theta_1.diff().diff()) - ((K_tw*K_ew)/R_mw + b_mw) * (theta_w.diff() - theta_1.diff()) + K_tw*V_w/R_mw

u_matrix = sp.Matrix([[-T_w],[T_w+T_b],[-T_b]])
tau_eq = tau+u_matrix

In [12]:
u_v = sp.Matrix([[V_w],[V_b]])
M, C, G, W = get_EoM_from_T(tau_eq,qdd,g, u_v)

In [13]:
linearlize_eq = {sp.sin(theta_1-theta_2):theta_1-theta_2, sp.cos(theta_1-theta_2):1,sp.sin(theta_2):theta_2, sp.cos(theta_2):1,sp.sin(theta_1):theta_1, sp.cos(theta_1):1, theta_1.diff()**2:0, theta_2.diff()**2:0}
Ml = M.subs(linearlize_eq)
Cl = C.subs(linearlize_eq)
Gl = G.subs(linearlize_eq)
Wl = W.subs(linearlize_eq)
Ml

Matrix([
[1.0*I_w + 1.0*J_mw + 1.0*m_1*r**2 + 1.0*m_b*r**2 + 1.0*m_w*r**2,            -1.0*J_mw + 1.0*L_1*m_b*r + 1.0*L_1c*m_1*r,                    1.0*L_b*m_b*r],
[                     -1.0*J_mw + 1.0*L_1*m_b*r + 1.0*L_1c*m_1*r, 1.0*I_1 + J_mb*n**2 + J_mw + L_1**2*m_b + L_1c**2*m_1,         -J_mb*n**2 + L_1*L_b*m_b],
[                                                  1.0*L_b*m_b*r,                              -J_mb*n**2 + L_1*L_b*m_b, 1.0*I_b + J_mb*n**2 + L_b**2*m_b]])

In [26]:
param = {L_b : 0.500417013226504,I_b :0.572443057472749,m_b: 19.245,I_1: 0.00614941, I_w:0.004806909,  m_1:2.786,   m_w:2.292, r:0.069,L_1c:0.171, L_1: 0.171,  g:9.81, n:250, J_mb: 3.33*10e-6, K_tb: 0.0276, K_eb:0.0276, b_mb:0.005, R_mb: 0.386,J_mw:0.004, K_tw:0.0519, K_ew:0.0519, b_mw: 2.7066*10e-4, R_mw:0.3}
Mlp = msubs(Ml, param)
Clp = msubs(Cl, param)
Glp = msubs(Gl, param)
Wlp = msubs(Wl, param)

Mlp_inv = Mlp.inv()
qdd_rhs_A = Mlp_inv*(-Clp -Glp)
qdd_rhs_B = Mlp_inv*Wlp*u

X = q.col_join(qd)
Xd_A = qd.col_join(qdd_rhs_A)
Xd_B = qd.col_join(qdd_rhs_B)
U = u

A = Xd_A.jacobian(X)
B = Xd_B.jacobian(U)
C = X.jacobian(X)
D = X.jacobian(U)

In [27]:
A

Matrix([
[0,                 0,                 0,                  1,                 0,                 0],
[0,                 0,                 0,                  0,                 1,                 0],
[0,                 0,                 0,                  0,                 0,                 1],
[0, -119.885647884584, -268.556484992699, -0.386676187141497,  175.288812595366, -174.902136408225],
[0,  26.6653097807229,  31.2143519137507, 0.0463371372467695, -170.513634390455,  170.467297253209],
[0,  12.2105341148243,  38.3372747315974, 0.0370774841361725,  32.8229783492034, -32.8600558333396]])

In [28]:
B

Matrix([
[                 0,                 0],
[                 0,                 0],
[                 0,                 0],
[  5.72471227743224,  7.17344631745327],
[-0.686017880900886, -6.99155556838434],
[-0.548929403229514,  1.34772422652862]])

In [38]:
I_bx, I_fx, I_fy, m_all, w, l, l_f, f_d, T_gb, g, r, L = sp.symbols('I_bx, I_fx, I_fy, m_all, w, l, l_f, f_d, T_gb, g, r, L')
K_t, K_e, R_m, J_m, V, n, f, b_m = sp.symbols('K_t, K_e, R_m, J_m, V, n, f, b_m')
theta_R, theta_gb = dynamicsymbols('theta_R, theta_gb')

theta_Rd = theta_R.diff()
theta_Rdd = theta_Rd.diff()
theta_gbd = theta_gb.diff()
theta_gbdd = theta_gbd.diff()

q = sp.Matrix([[theta_R], [theta_gb]])
qd = q.diff()
qdd = qd.diff()

u = sp.Matrix([T_gb])

L = I_fy * w / 2

T_gb = -n**2 * J_m * theta_gbdd - n**2 * (K_e*K_t/R_m + b_m)*theta_gbd + (n * K_t * V)/R_m

tau = sp.Matrix([[I_bx*theta_Rdd - m_all*g*l*sp.sin(theta_R) + 2*L*theta_gbd*sp.cos(theta_gb)],
                [I_fx*theta_gbdd -  L*theta_Rd*sp.cos(theta_gb) - T_gb]])
eq_point = {sp.sin(theta_gb):theta_gb, sp.cos(theta_gb):1,sp.sin(theta_R):theta_R, sp.cos(theta_R):1,theta_gbd**2:0 ,theta_Rd**2:0}

tau_eq = sp.simplify(tau.subs(eq_point))


In [39]:
tau_eq

Matrix([
[                                                                                                       I_bx*Derivative(theta_R(t), (t, 2)) + I_fy*w*Derivative(theta_gb(t), t) - g*l*m_all*theta_R(t)],
[(-K_t*V*n + R_m*(2*I_fx*Derivative(theta_gb(t), (t, 2)) - I_fy*w*Derivative(theta_R(t), t) + 2*J_m*n**2*Derivative(theta_gb(t), (t, 2)))/2 + n**2*(K_e*K_t + R_m*b_m)*Derivative(theta_gb(t), t))/R_m]])

In [40]:
u_v = sp.Matrix([V])

In [41]:
Ml, Cl, Gl, Wl = get_EoM_from_T(tau_eq,qdd,g,u_v)

In [42]:
Mlp_inv = Ml.inv()
qdd_rhs_A = Mlp_inv*(-Cl -Gl)
qdd_rhs_B = Mlp_inv*Wl*u

X = q.col_join(qd)
Xd_A = qd.col_join(qdd_rhs_A)
Xd_B = qd.col_join(qdd_rhs_B)
U = u

A = Xd_A.jacobian(X)
B = Xd_B.jacobian(U)
C = X.jacobian(X)
D = X.jacobian(U)

In [43]:
sp.simplify(A)

Matrix([
[             0, 0,                            1,                                                 0],
[             0, 0,                            0,                                                 1],
[g*l*m_all/I_bx, 0,                            0,                                      -I_fy*w/I_bx],
[             0, 0, I_fy*w/(2*(I_fx + J_m*n**2)), -n**2*(K_e*K_t + R_m*b_m)/(R_m*(I_fx + J_m*n**2))]])

In [28]:
B

Matrix([
[                            0],
[                            0],
[                            0],
[K_t*n/(R_m*(I_fx + J_m*n**2))]])

In [29]:
flywheel_ang_vel = (1500 * 2 * np.pi)/60 
# I_bx:9.448246482
param = {w:flywheel_ang_vel, I_fx:0.006235, I_fy:0.0119107, I_bx:0.458398057,  m_all:15, l: 0.5,  g:9.81, n:6, J_m: 0.08, K_e:0.88, K_t: 0.88, R_m:0.33, b_m:0}#22.031

Mlp = msubs(Ml, param)
Clp = msubs(Cl, param)
Glp = msubs(Gl, param)
Wlp = msubs(Wl, param)

Mlp_inv = Mlp.inv()
qdd_rhs_A = Mlp_inv*(-Clp -Glp)
qdd_rhs_B = Mlp_inv*Wlp*u

X = q.col_join(qd)
Xd_A = qd.col_join(qdd_rhs_A)
Xd_B = qd.col_join(qdd_rhs_B)
U = u

A = Xd_A.jacobian(X)
B = Xd_B.jacobian(U)
C = X.jacobian(X)
D = X.jacobian(U)

In [30]:
A

Matrix([
[               0, 0,                 1,                 0],
[               0, 0,                 0,                 1],
[160.504607025418, 0,                 0, -4.08144919548731],
[               0, 0, 0.324112274460603, -29.2699658898184]])

In [31]:
B

Matrix([
[               0],
[               0],
[               0],
[5.54355414579894]])